# Tratamento dos Dados de Exportação e Importação dos Estados Brasileiros

*Execute este arquivo no [![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Bug-Busters-F/alfalog/blob/feat/notebooks/tratamento-dados/Tratamento%20dos%20Dados%20de%20Exporta%C3%A7%C3%A3o%20e%20Importa%C3%A7%C3%A3o%20dos%20Estados%20Brasileiros.ipynb)*.

Leitura e limpeza dos dados abertos do Ministério do Desenvolvimento, Indústria, Comércio e Serviços.

Este notebook pertence projeto [Alfalog](https://github.com/Bug-Busters-F/alfalog) encontrado neste [link](https://github.com/Bug-Busters-F/alfalog).

O script abaixo irá realizar a leitura e limpeza dos dados do COMEX. Para isso, irá gerar dois arquivos CSV, um de exportação e um de importação, que serão salvos no Google Drive no diretório configurado em `DRIVE_TARGET`.


In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from pathlib import Path
from google.colab import drive

DRIVE_TARGET = "comex_data"

drive.mount('/content/drive')
drive_path = Path(f"/content/drive/MyDrive/{DRIVE_TARGET}")
drive_path.mkdir(parents=True, exist_ok=True)

base_url = "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/"
tables_url = "https://balanca.economia.gov.br/balanca/bd/tabelas/"

def download_csv(url):
    try:
        response = requests.get(url)
        return pd.read_csv(StringIO(response.text), sep=';', on_bad_lines='warn')
    except UnicodeDecodeError:
        return pd.read_csv(StringIO(response.text), sep=';', on_bad_lines='skip')

def process_comercio_exterior(tipo='EXP', anos=None):
    tipo = tipo.upper()
    linhas_base = 0
    if tipo not in ['EXP', 'IMP']:
        raise ValueError("Tipo deve ser 'EXP' (exportação) ou 'IMP' (importação)")

    if anos is None:
        from datetime import datetime
        ano_final = datetime.now().year
        ano_inicial = ano_final - 11
        anos = range(ano_inicial, ano_final)
    elif isinstance(anos, int):
        anos = [anos]
    elif isinstance(anos, (list, tuple)):
        pass
    else:
        raise ValueError("Anos deve ser um inteiro, lista ou tupla")

    print("Baixando tabelas de relacionamento...")
    paises = download_csv(tables_url + "PAIS.csv")[['CO_PAIS', 'NO_PAIS']]
    ufs = download_csv(tables_url + "UF.csv")[['SG_UF', 'NO_UF']]
    vias = download_csv(tables_url + "VIA.csv")[['CO_VIA', 'NO_VIA']]
    urf = download_csv(tables_url + "URF.csv")[['CO_URF', 'NO_URF']]
    ncm = download_csv(tables_url + "NCM.csv")[['CO_NCM', 'NO_NCM_POR']]

    all_data = []
    total_removed = {
        'column_filter': 0,
        'unit_filter': 0,
        'country_filter': 0,
        'uf_filter': 0,
        'via_filter1': 0,
        'via_filter2': 0,
        'qt_estat_filter': 0,
        'vl_fob_filter': 0,
        'ncm_freq_filter': 0,
        'pais_freq_filter': 0,
        'frete_filter': 0,
        'seguro_filter': 0,
        'outlier_filter': 0,
        'merge_loss': 0
    }

    for year in anos:
        print(f"\nProcessando ano {year}...")
        try:
            url = f"{base_url}{tipo}_{year}.csv"
            df = download_csv(url)
            initial_rows = len(df)
            linhas_base += initial_rows
            print(f"Linhas iniciais: {initial_rows}")
            before = len(df)
            expected_columns = [
                'CO_ANO', 'CO_MES', 'CO_NCM', 'CO_UNID', 'CO_PAIS', 'SG_UF_NCM',
                'CO_VIA', 'CO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB'
            ]

            before = len(df)
            df = df[df["CO_UNID"] != 18].copy()
            total_removed['unit_filter'] += (before - len(df))

            before = len(df)
            df = df[~df["CO_PAIS"].isin([0, 990, 994, 995, 997, 998, 999])].copy()
            total_removed['country_filter'] += (before - len(df))

            before = len(df)
            df = df[~df["SG_UF_NCM"].isin(["ND", "ZN", "ED", "RE", "MN", "CB", "EX"])].copy()
            total_removed['uf_filter'] += (before - len(df))

            before = len(df)
            df = df[~df["CO_VIA"].isin([0, 8, 10, 11, 12, 13, 99])].copy()
            total_removed['via_filter1'] += (before - len(df))

            before = len(df)
            vias_terrestres = [7, 6]
            paises_fronteira = [63, 97, 169, 337, 325, 586, 589, 770, 845, 850]
            df = df[~((df["CO_VIA"].isin(vias_terrestres)) & (~df["CO_PAIS"].isin(paises_fronteira)))].copy()
            total_removed['via_filter2'] += (before - len(df))

            before = len(df)
            df = df[df["QT_ESTAT"] != 0].copy()
            total_removed['qt_estat_filter'] += (before - len(df))

            before = len(df)
            df = df[df["VL_FOB"] != 0].copy()
            total_removed['vl_fob_filter'] += (before - len(df))

            before = len(df)
            ncms_frequentes = df["CO_NCM"].value_counts()[df["CO_NCM"].value_counts() > 5].index
            df = df[df["CO_NCM"].isin(ncms_frequentes)].copy()
            total_removed['ncm_freq_filter'] += (before - len(df))

            before = len(df)
            paises_frequentes = df["CO_PAIS"].value_counts()[df["CO_PAIS"].value_counts() > 10].index
            df = df[df["CO_PAIS"].isin(paises_frequentes)].copy()
            total_removed['pais_freq_filter'] += (before - len(df))

            before = len(df)
            Q1 = df["KG_LIQUIDO"].quantile(0.25)
            Q3 = df["KG_LIQUIDO"].quantile(0.75)
            IQR = Q3 - Q1
            df = df[(df["KG_LIQUIDO"] >= Q1 - 1.5*IQR) & (df["KG_LIQUIDO"] <= Q3 + 1.5*IQR)].copy()
            total_removed['outlier_filter'] += (before - len(df))

            df['ANO'] = year
            all_data.append(df)
            print(f"Linhas restantes após todos os filtros: {len(df)}")

        except Exception as e:
            print(f"Erro no ano {year}: {str(e)}")

    if not all_data:
        raise ValueError("Nenhum dado válido foi processado")

    final_df = pd.concat(all_data, ignore_index=True)

    before_merge = len(final_df)
    final_df = final_df.merge(paises, on='CO_PAIS', how='left')
    final_df = final_df.merge(ufs, left_on='SG_UF_NCM', right_on='SG_UF', how='left')
    final_df = final_df.merge(vias, on='CO_VIA', how='left')
    final_df = final_df.merge(urf, on='CO_URF', how='left')
    final_df = final_df.merge(ncm, on='CO_NCM', how='left')

    cols_order = [
        'ANO', 'CO_MES', 'NO_NCM_POR', 'NO_PAIS', 'SG_UF_NCM', 'NO_UF',
        'NO_VIA', 'NO_URF', 'KG_LIQUIDO', 'CO_UNID', 'VL_FOB'
    ]
    final_df["NO_VIA"] = final_df["NO_VIA"].replace({
        "MARÍTIMA": "AQUÁTICA", "FLUVIAL": "AQUÁTICA", "LACUSTRE": "AQUÁTICA"
    })
    final_df = final_df[cols_order].rename(columns={'SG_UF_NCM': 'SG_UF'})

    output_file = drive_path / f'dados_comex_{tipo}_{min(anos)}_{max(anos)}.csv'
    final_df.to_csv(output_file, index=False)

    # Resumo estatístico consolidado
    print("\nRESUMO CONSOLIDADO DE FILTRAGEM (TODOS OS ANOS):")
    print("="*50)
    print(f"Total de linhas iniciais: {linhas_base}")
    print(f"Total de linhas finais: {len(final_df)}")
    print(f"Total geral de linhas removidas: {linhas_base - len(final_df)}")
    print("\nDetalhe por tipo de filtro:")
    print("-"*50)
    print(f"Filtro de unidades (CO_UNID != 18): {total_removed['unit_filter']}")
    print(f"Filtro de países inválidos: {total_removed['country_filter']}")
    print(f"Filtro de UFs inválidas: {total_removed['uf_filter']}")
    print(f"Filtro de vias inválidas: {total_removed['via_filter1']}")
    print(f"Filtro de vias terrestres inconsistentes: {total_removed['via_filter2']}")
    print(f"Filtro de quantidade estatística zero: {total_removed['qt_estat_filter']}")
    print(f"Filtro de valor FOB zero: {total_removed['vl_fob_filter']}")
    print(f"Filtro de NCMs pouco frequentes: {total_removed['ncm_freq_filter']}")
    print(f"Filtro de países pouco frequentes: {total_removed['pais_freq_filter']}")
    if total_removed['frete_filter'] > 0:
        print(f"Filtro de frete inconsistente: {total_removed['frete_filter']}")
    if total_removed['seguro_filter'] > 0:
        print(f"Filtro de seguro inconsistente: {total_removed['seguro_filter']}")
    print(f"Filtro de outliers: {total_removed['outlier_filter']}")
    print("="*50)

    print(f"\nProcesso concluído! Arquivo salvo em: {output_file}")
    return final_df
df = process_comercio_exterior()
df2 = process_comercio_exterior('IMP')
df.head(10)
# df2.head(10)

MessageError: Error: credential propagation was unsuccessful